# Weather data analysis


In [65]:
import pandas as pd
import numpy as np
import json

First load the sample dataframe and extract each column info

In [66]:
v_dict = {}
v_data = json.load(open('data/variables.json', encoding='UTF-8'))
for var in v_data:
    v_dict[var['codi_variable']] = var['acronim']

In [67]:
df = pd.read_csv('data/prueba.csv')

cols = list(df.columns)
cols[0] = 'id'

for i in range(1, len(cols)):
    cols[i] = v_dict[cols[i]]
    
df.columns = cols

Eliminar las columas que hagan referencia al viento en 2 y 6 metros, en total 4 columnas.

In [68]:
two_meters = ['VV2', 'DV2', 'VVx2', 'DVVx2']
six_meters = ['VV6', 'DV6', 'VVx6', 'DVVx6']
df.drop(columns=two_meters, inplace=True)
df.drop(columns=six_meters, inplace=True)

In [69]:
df = df.replace(-1, np.nan)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 755383 entries, 0 to 755382
Data columns (total 19 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   id        755383 non-null  object 
 1   PPTx1min  697892 non-null  float64
 2   HRx       734708 non-null  float64
 3   VV10      357885 non-null  float64
 4   DV10      357882 non-null  float64
 5   T         734487 non-null  float64
 6   HR        734787 non-null  float64
 7   P         307048 non-null  float64
 8   PPT       743100 non-null  float64
 9   RS        583389 non-null  float64
 10  GNEU      73663 non-null   float64
 11  Tx        734394 non-null  float64
 12  Tn        734358 non-null  float64
 13  HRn       734706 non-null  float64
 14  VVx10     357816 non-null  float64
 15  DVVx10    357816 non-null  float64
 16  RN        44838 non-null   float64
 17  Px        307030 non-null  float64
 18  Pn        307031 non-null  float64
dtypes: float64(18), object(1)
memory usage: 109.

In [73]:
missing_percent = df.isnull().sum() * 100 / len(df)
missing_percent.sort_values(ascending=False)

RN          94.064203
GNEU        90.248258
Px          59.354394
Pn          59.354261
P           59.352011
DVVx10      52.631182
VVx10       52.631182
DV10        52.622445
VV10        52.622047
RS          22.769112
PPTx1min     7.610841
Tn           2.783356
Tx           2.778590
T            2.766279
HRn          2.737287
HRx          2.737022
HR           2.726564
PPT          1.626063
id           0.000000
dtype: float64

Vemos que la RN (Irradiància neta) no esta presente en muchos casos y tampoco la presencia de nieve (GNEU). Esta última puede ser debido a que en Cataluña no nieva en exceso salvo en areas de montaña. Ademas no es significativa en el periodo de incendios a estudiar (primavera verano).

Las variables que tienen menos de un 10% de valores sin rellenar se deben a que son tests automaticos?

In [85]:
df['id'].astype(str)
df['station'] = df['id'].str[:2]
df['date'] = pd.to_datetime(df['id'].str[2:], format='%d%m%y%H%M')
df.head()


,id,PPTx1min,HRx,VV10,DV10,T,HR,P,PPT,RS,...,Tx,Tn,HRn,VVx10,DVVx10,RN,Px,Pn,station,date
0,XQ0504172300,0.0,61.0,NaN,NaN,NaN,60.0,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,XQ,2017-04-05 23:00:00
1,XR0504172300,0.0,89.0,NaN,NaN,5.5,89.0,NaN,0.0,0.0,...,5.7,5.3,88.0,NaN,NaN,NaN,NaN,NaN,XR,2017-04-05 23:00:00
2,XS0504172300,0.0,81.0,1.7,295.0,10.5,78.0,NaN,0.0,0.0,...,10.7,10.2,76.0,4.1,286.0,NaN,NaN,NaN,XS,2017-04-05 23:00:00
3,XT0504172300,0.0,93.0,0.7,265.0,4.8,92.0,939.8,0.0,0.0,...,5.3,4.3,89.0,2.2,285.0,NaN,940.0,939.7,XT,2017-04-05 23:00:00
4,XU0504172300,0.0,96.0,0.2,269.0,8.7,96.0,NaN,0.0,0.0,...,8.8,8.6,95.0,0.9,228.0,NaN,NaN,NaN,XU,2017-04-05 23:00:00


Ahora mirare la distribución de las distintas estaciones, algunas no apareceran porque los datos son solamente los ultimos 10millones de observaciones.


In [94]:
df.station.value_counts()

XQ    4094
UB    4094
U7    4094
U9    4094
VA    4094
      ... 
XI    4093
XL    4093
XK    4093
XJ    4093
YE    2095
Name: station, Length: 185, dtype: int64

In [95]:
stations = json.load(open('data/stations.json', encoding='UTF-8'))
len(stations)

233